🎯 CONTEXTO DEL PROYECTO

¿Dónde estamos ahora?
Hemos completado exitosamente la Fase 1|2: EDA y Limpieza. Tenemos un dataset limpio con 1,470 empleados y 30 variables listas para analizar.

¿Cuál es el objetivo del negocio?

ABC Corporation quiere:

✅ Reducir la rotación de empleados (attrition)

✅ Mejorar la satisfacción laboral

✅ Identificar factores clave que influyen en ambos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('../data/processed/hr_processed.csv')

Fase 2: Análisis Exploratorio
│
├── 2.1 Análisis de Rotación (Attrition)
│   ├── ¿Cuál es la tasa de rotación actual?
│   ├── ¿Qué características tienen los empleados que se van?
│   └── ¿Qué patrones vemos en la rotación?
│
├── 2.2 Análisis de Satisfacción Laboral
│   ├── ¿Cómo están de satisfechos los empleados?
│   ├── ¿Qué variables se relacionan con la satisfacción?
│   └── ¿Hay diferencias por departamento/rol?
│
├── 2.3 Análisis de Factores Demográficos
│   ├── Edad, género, estado civil
│   ├── Distancia desde casa
│   └── Educación y field
│
├── 2.4 Análisis de Factores Laborales
│   ├── Salario y compensación
│   ├── Horas extra (overtime)
│   ├── Balance vida-trabajo
│   └── Años en la empresa
│
└── 2.5 Hallazgos y Recomendaciones
    ├── Top 5 factores de riesgo de rotación
    ├── Segmentos críticos de empleados
    └── Recomendaciones accionables

Paso 1: Análisis Univariado

Entender cada variable individualmente.

Preguntas clave:

- ¿Cuál es la distribución de edad?

- ¿Cuántos empleados hay por departamento?

- ¿Cuál es el salario promedio?

- ¿Qué porcentaje hace overtime?

Herramientas:




In [10]:
df['columns'].value_counts()
df['columns'].describe()
Histogramas, barplots

KeyError: 'columns'

Paso 2: Análisis Bivariado

Relacionar variables con la variable objetivo (attrition).

Preguntas clave:

- ¿La rotación es mayor en empleados jóvenes?

- ¿Los empleados que hacen overtime se van más?

- ¿El salario bajo aumenta la rotación?

- ¿La satisfacción laboral predice la rotación?

In [ ]:
#Tablas cruzadas: 
pd.crosstab()
#Comparación de grupos: 
df.groupby('attrition')['age'].mean()
#Visualizaciones: 
boxplots, barplots agrupados

Paso 3: Análisis Multivariado

Ver patrones complejos entre múltiples variables.

Preguntas clave:

- ¿Hay perfiles de empleados en riesgo?

- ¿Qué combinaciones de factores aumentan la rotación?

- ¿Existen correlaciones entre satisfacción, salario y balance?

Herramientas:

Matrices de correlación

Heatmaps

Análisis de segmentación

📊 ESQUEMA DE CONSULTAS (QUERIES) SUGERIDAS

GRUPO 1: Análisis de Rotación

In [ ]:
# 1. Tasa general de rotación
tasa_rotacion = (df['attrition'] == 'Yes').mean() * 100

# 2. Rotación por departamento
rotacion_dept = df.groupby('department')['attrition'].apply(
    lambda x: (x == 'Yes').mean() * 100
)

# 3. Rotación por nivel de trabajo
rotacion_nivel = df.groupby('job_level')['attrition'].apply(
    lambda x: (x == 'Yes').mean() * 100
)

# 4. Perfil de empleados que se van vs. se quedan
comparacion = df.groupby('attrition').agg({
    'age': 'mean',
    'monthly_income': 'mean',
    'years_at_company': 'mean',
    'job_satisfaction': lambda x: (x == 'Muy satisfecho').mean()
})

GRUPO 2: Análisis de Satisfacción

In [ ]:
# 5. Distribución de satisfacción laboral
satisfaccion_dist = df['job_satisfaction'].value_counts(normalize=True)

# 6. Relación satisfacción-rotación
satisfaccion_attrition = pd.crosstab(
    df['job_satisfaction'], 
    df['attrition'], 
    normalize='index'
) * 100

# 7. Satisfacción por departamento
satisfaccion_dept = pd.crosstab(
    df['department'], 
    df['job_satisfaction'], 
    normalize='index'
) * 100

# 8. Factores que se correlacionan con satisfacción
# (necesitaremos codificar job_satisfaction a numérico primero)

GRUPO 3: Factores de Riesgo

In [ ]:
# 9. Empleados con overtime y rotación
overtime_attrition = pd.crosstab(
    df['over_time'], 
    df['attrition'], 
    normalize='index'
) * 100

# 10. Balance vida-trabajo y rotación
balance_attrition = pd.crosstab(
    df['work_life_balance'], 
    df['attrition'], 
    normalize='index'
) * 100

# 11. Empleados jóvenes (< 30 años) vs mayores
df['grupo_edad'] = pd.cut(df['age'], bins=[0, 30, 40, 50, 100], 
                           labels=['<30', '30-40', '40-50', '50+'])
edad_attrition = pd.crosstab(
    df['grupo_edad'], 
    df['attrition'], 
    normalize='index'
) * 100

# 12. Distancia al trabajo
distancia_attrition = df.groupby('attrition')['distance_from_home'].mean()

GRUPO 4: Análisis de Compensación

In [ ]:
# 13. Salario promedio por nivel y rotación
salario_analisis = df.groupby(['job_level', 'attrition'])['monthly_income'].mean().unstack()

# 14. Aumentos salariales (percent_salary_hike)
aumento_attrition = df.groupby('attrition')['percent_salary_hike'].mean()

# 15. Stock options y rotación
stock_attrition = pd.crosstab(
    df['stock_option_level'], 
    df['attrition'], 
    normalize='index'
) * 100

Ejemplo 1: ¿El overtime afecta la rotación?

In [ ]:

# Cargar datos
df = pd.read_csv('hr_processed.csv')

# Análisis
overtime_vs_attrition = pd.crosstab(
    df['over_time'], 
    df['attrition'], 
    normalize='index'
) * 100

print(overtime_vs_attrition)

# Visualización
overtime_vs_attrition.plot(kind='bar', stacked=False)
plt.title('Rotación según Overtime')
plt.ylabel('Porcentaje (%)')
plt.xlabel('¿Hace Overtime?')
plt.legend(['Se queda', 'Se va'])
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

Interpretación esperada:

Si el % de rotación es mayor en "Yes" que en "No", el overtime es un factor de riesgo.

Cuantifica: "Los empleados con overtime tienen X% más probabilidad de irse"

Ejemplo 2: Perfil del empleado en riesgo

In [ ]:
# Comparar características entre los que se van y se quedan
perfil_riesgo = df.groupby('attrition').agg({
    'age': ['mean', 'median'],
    'monthly_income': ['mean', 'median'],
    'years_at_company': ['mean', 'median'],
    'distance_from_home': 'mean',
    'num_companies_worked': 'mean'
}).round(2)

print(perfil_riesgo)

Interpretación esperada:

"Los empleados que se van son en promedio X años más jóvenes"

"Ganan $X menos al mes"

"Han trabajado en Y empresas antes"